# Load and preprocess 1976 data

Time to start looking at other years!

---

In [1]:
%pylab --no-import-all inline
import pandas as pd
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


## Load the data.

---

If this fails, be sure that you've saved your own data in the prescribed location, then retry.

In [7]:
file = "../data/NES1976.dta"  # Their filename, not mine.
df_rawest = pd.read_stata(file)

ValueError: Version of given Stata file is not 104, 105, 108, 113 (Stata 8/9), 114 (Stata 10/11), 115 (Stata 12), 117 (Stata 13), or 118 (Stata 14)